Now we need to perform an inverse text normalization to obtain examples to train an LLM on.

In [ ]:
import json
from tqdm.notebook import tqdm
import random

In [ ]:
from collections import defaultdict, Counter

In [ ]:
# INPUT_PATH = "librusec.jsonl"
# OUTPUT_PATH = "librusec_pairs.jsonl"

INPUT_PATH = "pikabu.jsonl"
OUTPUT_PATH = "pikabu_pairs.jsonl"

In [ ]:
data = []
with open(INPUT_PATH) as f:
    for line in tqdm(f):
        data.append(json.loads(line.strip()))
random.choice(data)

I end up using `nemo_text_processing` as it is fast.
This part is vital so better use another one here next time.

Seems ok for a first attempt anyway.

In [ ]:
from nemo_text_processing.inverse_text_normalization.inverse_normalize import InverseNormalizer
inverse_normalizer = InverseNormalizer(lang='ru')

Make some balancing here.
First, perform a rough distribution check.

In [ ]:
counter = Counter()
for elem in tqdm(data):
    for text in elem["texts"]:
        for match in text["matches"]:
            counter[match["number"]] += 1
sum(counter.values()), counter

Seems too much of numbers less than 10.
So we strip a part of the most common numbers.

In [ ]:
from multiprocessing import Process, Queue
from multiprocessing import Pool


queue = Queue()
occurances = Counter()
occurances_limit = sum(counter.values()) / 25
print(f"{occurances_limit=}")


def inverse_normalize(elem):
    result = []
    for i_text, text in enumerate(elem["texts"]):
        matches = [match for match in text["matches"] if occurances[match["number"]] < occurances_limit]
        if not matches:
            continue
        occurances.update((_["number"] for _ in matches))
        itn = inverse_normalizer.inverse_normalize(text["text"], verbose=False)
        if itn == text["text"]:
            continue
        result.append({
            "tn": text["text"],
            "itn": itn,
            "orig_index": elem["index"],
            "text_index": i_text
        })
    return result


def process(elem):
    for result in inverse_normalize(elem):
        queue.put(result)


def write(queue):
    f = open(OUTPUT_PATH, "w")
    while True:
        item = queue.get()
        if item is None:
            return
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")
    f.close()


writer = Process(target=write, args=(queue,))
writer.start()
with Pool(10) as p:
    p.map(process, tqdm(data))
    queue.put(None)
p.join()
writer.join()